# PyPerceive to select and load Percept recordings 

## 0a. Loading default packages and functions

In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np

import scipy
import matplotlib.pyplot as plt
from scipy import signal

import openpyxl
from openpyxl import Workbook, load_workbook
import xlrd

#mne
import mne_bids
import mne
from mne.time_frequency import tfr_morlet 

from importlib import reload          


#### check package versions

developed with:
- Python sys 3.10.8
- pandas 1.5.1
- numpy 1.23.4
- mne_bids 0.11.1
- mne 1.2.3

In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)


## 0b. Loading pyPerceive functions

In [ ]:
def add_and_set_code_folder_in_notebook():
    """
    while working in the local pyPerceive repo,
    find and set path to the PyPerceive code folder

    use function in notebook first, to locate the local
    repo and enable import of pyPerceive functions
    """
    project_path = os.getcwd()

    while project_path[-10:] != 'PyPerceive':
        project_path = os.path.dirname(project_path)

    code_path = os.path.join(project_path, 'code')
    sys.path.append(code_path)

    # change directory to code path
    os.chdir(code_path)
    
    return print(f'working dir set to: {code_path}')


In [ ]:
## MAIN FUNCTION FOR DATA IMPORT

# change working directory to ensure correct loading of own functions
add_and_set_code_folder_in_notebook()

# import main class to work with
from PerceiveImport.classes import main_class


In [ ]:
## IMPORT ALL SUB CLASSES AND FUNCTIONS FOR DEBUGGING
from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class, chronic_class
)
import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers

JSON Anonymize

In [ ]:
import PerceiveImport.methods.find_folders as find_folders 
import utils.anonymise_jsons as anonymise_jsons
import PerceiveImport.methods.metadata_helpers as metadata_helpers 

importlib.reload(anonymise_jsons)
importlib.reload(find_folders)
importlib.reload(metadata_helpers)

In [ ]:
anonymise_jsons.anonymise_jsons()

In [ ]:
metadata_helpers.get_terminology(
    key = "session"
)

## 1. Test Data Loading for Streaming and Survey

In [ ]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further

importlib.reload(run_class)
importlib.reload(load_rawfile)

# sub030 = main_class.PerceiveData(
#     sub = "030", 
#     incl_modalities=['streaming', ],  # 'survey', 'indefiniteStreaming'
#     incl_session = ["fu12m"],
#     incl_condition =['m1s0', 'm1s1',],
#     incl_task = ["rest", "ChangingPositionMoveArt"],
#     incl_contact = ["RingL", "RingR"],
#     import_json=False,
#     warn_for_metaNaNs=True,
#     allow_NaNs_in_metadata=True,
#     # use_bids=True,  # TODO: add to functionality
# )

dat = main_class.PerceiveData(
    sub = "040", 
    incl_modalities=['streaming', 'survey'],  # 'survey', 'indefiniteStreaming'
    incl_session = ["fu12m"],
    incl_condition =['m0s0', 'm1s0', 'm1s1',],
    incl_task = ["rest", ],  # "ChangingPositionMoveArt"
    # incl_contact = ["RingL", "RingR"],
    import_json=False,
    warn_for_metaNaNs=True,
    allow_NaNs_in_metadata=True,
    # use_bids=True,  # TODO: add to functionality
)

In [ ]:
dat.streaming.fu12m.m1s1.rest.run1.data.ch_names

dat.survey.fu12m.m1s0.rest.SegmIntraL.run1.data.ch_names

In [ ]:
json_sub030.streaming.fu12m.m1s1.rest.run1.json

In [ ]:
# WHEN IMPORT FROM .MAT (import_json=False)
sub030.streaming.fu12m.m1s0.rest.run1.data  # MNE RawArray, 


sub030.streaming.fu12m.m1s0.rest.run1.data.get_data().shape

print(sub030.streaming.fu12m.m1s0.rest.run1.data.ch_names)

plt.plot(sub030.streaming.fu12m.m1s0.rest.run1.data.get_data()[0, :])
plt.show()

In [ ]:
secs = sub030.streaming.fu12m.m1s0.rest.run1.data.times.copy()
type(secs)
start_time = 85  # seconds from start
end_time = 92

index_start = np.where(secs == start_time)[0][0]
index_end = np.where(secs == end_time)[0][0]

print(index_start, index_end)

mov_part = sub030.streaming.fu12m.m1s0.rest.run1.data.get_data()[0, index_start:index_end]

type(mov_part)

## 2. Test Data Loading for Chronic

In [ ]:
# in case of debugging
from PerceiveImport.methods import extract_chronic_timeline_samples as extract_chronic

In [ ]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further
importlib.reload(find_folders)
importlib.reload(extract_chronic)
importlib.reload(chronic_class)
importlib.reload(metaHelpers)
importlib.reload(main_class)
importlib.reload(modality_class)

dat = main_class.PerceiveData(
    sub = "080", 
    incl_modalities=['chronic'],
    import_json=True,
    warn_for_metaNaNs=False,
)


In [ ]:
# print(len(dat.chronic.events))

sum([e.contains_LFP for e in dat.chronic.events])

In [ ]:
# first take all unique events with LFP data
uniq_lfp_times, uniq_lfp_idx = np.unique([e.time for e in dat.chronic.events
                                  if e.contains_LFP],  #
                                 return_index=True)
uniq_lfp_events = list(np.array(dat.chronic.events)[uniq_lfp_idx])

uniq_noLFP_times, uniq_noLFP_idx = np.unique([e.time for e in dat.chronic.events
                                              if not e.contains_LFP],  #
                                             return_index=True)
uniq_nolfp_events = list(np.array(dat.chronic.events)[uniq_noLFP_idx])

uniq_nolfp_events = [e for e in uniq_nolfp_events
                     if e.time not in uniq_lfp_times]


## 3. Direct access JSONs

TODO:
- finish SnapSHot extraction
- fix JSON import, extract recording order from excel and get metadata
    - use this to order the list of JSON data
    - then use correct and check missings on each data part

In [ ]:
# test brainsense with different files

# test snapshots and chronic
json_fname = 'Report_Json_Session_Report_20231123T141534.json'
j = load_rawfile.load_sourceJSON('047', json_fname)

json_fname = 'Report_Json_Session_Report_20231123T141637.json'
j2 = load_rawfile.load_sourceJSON('047', json_fname)

json_fname = 'Report_Json_Session_Report_20231123T141705.json'
j3 = load_rawfile.load_sourceJSON('047', json_fname)

In [ ]:
print(j.keys())
print(j2.keys())

In [ ]:
prc_data_codes = {
    'signal_test': 'CalibrationTests',
    'streaming': 'BrainSenseTimeDomain',
    'survey': 'LfpMontageTimeDomain',
    'indef_streaming': 'IndefiniteStreaming'
}

In [ ]:
import PerceiveImport.methods.timezone_handling as tz_handling

In [ ]:
importlib.reload(tz_handling)

In [ ]:
j[prc_data_codes['survey']]  # TO INTEGRATE

# check_and_correct_missings_in_lfp(j['BrainSenseLfp'][4])

In [ ]:
mod = 'streaming'

list_of_streamings = j[prc_data_codes[mod]]
n_streamings = len(list_of_streamings)


list_of_streamings2 = j2[prc_data_codes[mod]]

# n_exp_streamings = extract from metadata
# check whether n-streamings match metdata table 
# if n_streamings == n_streamings: ...

for dat in list_of_streamings2:
    print(dat)

In [ ]:
# list_of_streamings[0]['GlobalPacketSizes']

print(list_of_streamings[0].keys())

In [ ]:
# dat = list_of_streamings[0]
for i_dat, dat in enumerate(list_of_streamings):
    print(i_dat)
    new_lfp = load_rawfile.check_and_correct_missings_in_lfp(dat)
